# Reflexivity in a diffusion of innovations model

This code tries to replicate the diffusion of innovations model present in

> *Diffusion dynamics in small-world networks with heterogeneous consumers* from Delre, Sebastiano A., Jager, Wander and Janssen, Marco A., Computational and Mathematical Organization Theory, **13**, 2, 2007.

with some modifications.

It also adds new features that try to introduce reflexivity in it.

## 1. Initialization

### 1.1 External libraries

In [ ]:
from __future__ import division

import os

from ipyparallel import Client
import numpy as np
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
if not os.name == 'nt':
    matplotlib.rc('text', usetex=True)

In [ ]:
#plt.style.use('seaborn-paper')
sns.set_style("whitegrid")

### 1.2 Autoreload

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
# Client and DirectView for IPyparallel kernels
rc = Client()
dview = rc[:]

In [ ]:
%px %load_ext autoreload

In [ ]:
%px %autoreload 2

### 1.3 Internal code

In [ ]:
from algorithm import compute_run, generate_initial_conditions, generate_parameters
from plots import plot_adopters, plot_indexes

In [ ]:
# Import our algorithm here and in IPyparallel kernels
with rc[:].sync_imports():
    from algorithm import single_run

## 2. Running the model

### 2.1 Parameters

These are main parameters that control the evolution of the algorithm, and their corresponding variable in the article:

* Network randomness: $r$
* Average number of neighbors: $k$
* Initial proportion (or seed) of adopters: $\delta$
* Coefficient of social influence: $\beta$
* Threshold of adopters among neighbors: $h$
* Quality: $q$
* Total number of consumers: $N$
* Activation sharpness: $\phi$
* Critical mass of adopters: $M_{c}$

----

* Marketing effort: $e_{1}$
* Level: $L$

In [ ]:
parameters = dict(
    randomness = 0.01,
    number_of_neighbors = 15,
    initial_seed = 0.001,
    adopters_threshold = 0.8,
    social_influence = 0.7,
    quality = 0.5,
    number_of_consumers = 1000,
    activation_sharpness = 5,
    critical_mass = 0.25,
    marketing_effort = 0,
    level = 1
    #minimal_utility = 0.5,
)

### 2.2 Simulation

In [ ]:
data = compute_run(30, parameters, max_time=45, dview=dview)

## 3. Plots

### 3.1 2d plots

In [ ]:
plt.figure(figsize=(8, 6))
plot_adopters(data, 'k', parameters['number_of_neighbors'])

In [ ]:
# Plot the cumulative sum of adopters and non-adopters (the usual diffusion curve)
plt.figure(figsize=(8, 6))
plot_adopters(data, 'k', parameters['number_of_neighbors'], cumulative=True)

### 3.2 Logistic function form

This explores how the logistic function changes under different values of $k$ and $x_{0}$

In [ ]:
from sympy import plot, symbols, exp

In [ ]:
x = symbols('x')

In [ ]:
def symbolic_logistic(x, k, x0):
    return 1 / ( 1 + exp(-k * (x - x0)) )

In [ ]:
plot(symbolic_logistic(x, 25, 0.6), 1, (x, 0, 1))

In [ ]:
for i in np.arange(0, 0.5, 0.01):
    r = symbolic_logistic(i, 90, 0.5)
    if r > 0.005:
        print(i)
        break

### 3.3 Animations

In [ ]:
%matplolib qt

In [ ]:
import matplotlib.animation as animation

In [ ]:
G = generate_initial_conditions(1000, parameters)

In [ ]:
# Plot an animation of the evolution
fig = plt.figure()
positions = nx.spring_layout(G)
animation.FuncAnimation(fig, lambda i: animate(i, G, positions, parameters, test=False),
                        frames=500, interval=3, repeat=False)

## 4. Results

### 4.1 Varying adopters threshold

In [ ]:
adopters_parameters = dict(
    randomness = 0.01,
    number_of_neighbors = 15,
    initial_seed = 0.001,
    social_influence = 0.7,
    quality = 0.5,
    number_of_consumers = 1000,
    activation_sharpness = 25,
    critical_mass = 0.6,
    marketing_effort = 0,
    level = 1
)

In [ ]:
thresholds = [0.1, 0.5, 0.7, 0.9]
set_of_parameters = generate_parameters(adopters_parameters, 'adopters_threshold', thresholds)

In [ ]:
data = []
for p in set_of_parameters:
    p_data = compute_run(number_of_times=50, parameters=p, max_time=60, dview=dview)
    data.append(p_data)

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(9, 9), sharex=True, sharey=True)

for ax, d, t in zip(axes.flat, data, thresholds):
    plot_adopters(d, 'h', t, axis=ax, fontsize=14, cumulative=True)

f.tight_layout()

In [ ]:
if os.name == 'nt':
    figsize = (9, 9)
    fontsize = 11
else:
    figsize = (10, 10)
    fontsize = 15

fig = plt.figure(figsize=figsize)

outer_grid = gridspec.GridSpec(2, 2, hspace=0.15)

for i, d, t in zip(range(4), data, thresholds):
    inner_grid = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer_grid[i],
                                                  height_ratios = [1, 2.6],
                                                  hspace=0.02)
    ax_top = plt.Subplot(fig, inner_grid[0])
    ax_joint = plt.Subplot(fig, inner_grid[1])

    ax_joint.set_ylim(top=3000)
    if i == 0 or i == 1:
        plt.setp(ax_joint.get_xticklabels(), visible=False)
    if i == 1 or i == 3:
        plt.setp(ax_joint.get_yticklabels(), visible=False)

    fig.add_subplot(ax_joint)
    fig.add_subplot(ax_top, sharex=ax_joint)

    plot_adopters(d, 'h', t, axis=ax_joint, fontsize=fontsize, cumulative=True)
    plot_indexes(d, axis=ax_top, critical_mass=0.4, max_time=100)